In [33]:
import glob
import os
import random
import h5py
import pickle
import networkx as nx
import matplotlib.pyplot as plt
import torch

output_dir = r"C:\Users\uhewm\Desktop\ProjectHGT\simulation_chunks"
all_files = sorted(glob.glob(os.path.join(output_dir, "*.h5")))

for file in random.sample(all_files, 1):
    with h5py.File(file, "r") as f:
        grp = f["results"]
        graph_properties = pickle.loads(grp["graph_properties"][()])
        # Tensors erstellen
        nodes = torch.tensor(graph_properties[0])          # Shape [199]
        edges = torch.tensor(graph_properties[1])          # Shape [2, 198]
        coords = torch.tensor(graph_properties[2].T)       # Shape [2, 199]
        parental_nodes_hgt_events_corrected = grp.attrs["parental_nodes_hgt_events_corrected"]
        gene_absence_presence_matrix = grp.attrs["gene_absence_presence_matrix"] 

# Erstelle einen gerichteten Graphen
G = nx.DiGraph()

# Füge Knoten hinzu (optional mit Koordinaten als Attribut)
for i, node_id in enumerate(nodes.tolist()):
    G.add_node(node_id, core_distance = coords[:, i].tolist()[0], allele_distance = coords[:, i].tolist()[1])

# Füge Kanten hinzu
edge_list = edges.tolist()
for src, dst in zip(edge_list[0], edge_list[1]):
    G.add_edge(src, dst)

G = G.reverse()

# 1. Leaves identifizieren (Knoten ohne Nachfolger)
leaves = [n for n in G.nodes if G.out_degree(n) == 0]

# 2. Initialisiere Level-Dictionary
levels = {n: 0 for n in leaves}  # Leaves haben Level 0

# 3. Topologische Sortierung (damit Kinder vor Eltern behandelt werden)
for node in reversed(list(nx.topological_sort(G))):
    successors = list(G.successors(node))
    if successors:
        levels[node] = 1 + max(levels[s] for s in successors)

# 4. Level als Attribut setzen
nx.set_node_attributes(G, levels, "level")

G = G.reverse()

C:\Users\uhewm\AppData\Local\Temp\ipykernel_19520\1101100201.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edges = torch.tensor(graph_properties[1])          # Shape [2, 198]
C:\Users\uhewm\AppData\Local\Temp\ipykernel_19520\1101100201.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  coords = torch.tensor(graph_properties[2].T)       # Shape [2, 199]


In [2]:
print(graph_properties[2])
print(G.nodes)
#print(torch.tensor([1 if node in result["parental_nodes_hgt_events"] else 0 for node in graph_properties[0]]))
#print(torch.tensor([1 if node in result["parental_nodes_hgt_events_corrected"] else 0 for node in graph_properties[0]]))
print(parental_nodes_hgt_events_corrected)
print(gene_absence_presence_matrix)
print(graph_properties[1])

tensor([[0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.000

In [10]:
from pyvis.network import Network

# Falls nicht schon passiert:
net = Network(notebook=True, height="800px", width="100%")

# Voraussetzung: Du hast `parental_nodes_hgt_events_corrected` als Set oder Liste
# Und `gene_absence_presence_matrix` als dict oder Liste

for n in G.nodes:
    core = G.nodes[n].get('core_distance', 0)
    allele = G.nodes[n].get('allele_distance', 0)
    title = f"Core: {core:.2f}, Allele: {allele:.2f}"
    label = f"{n}\n({core:.2f}, {allele:.2f})"

    # Default-Farbe
    color = "lightblue"

    # Rote Knoten: parental HGT
    if n in parental_nodes_hgt_events_corrected:
        color = "red"

    # Grüne Knoten: node < 100 und gene == 1
    elif n < 100 and gene_absence_presence_matrix[n] == 1:
        color = "green"
    elif n < 100 and gene_absence_presence_matrix[n] == 0:
        color = "black"

    net.add_node(n, label=label, title=title, color=color)

# Kanten hinzufügen
for u, v in G.edges:
    net.add_edge(u, v)

# Anzeigen
net.show("graph.html")



graph.html


In [16]:
from pyvis.network import Network

# Netzwerk initialisieren mit hierarchischem Layout
net = Network(height="900px", width="100%", directed=True)
net.set_options("""
{
  "layout": {
    "hierarchical": {
      "enabled": true,
      "direction": "UD",
      "sortMethod": "directed",
      "levelSeparation": 100,
      "nodeSpacing": 150,
      "treeSpacing": 200
    }
  },
  "nodes": {
    "shape": "dot",
    "size": 12,
    "font": { "size": 16 }
  },
  "edges": {
    "arrows": {
      "to": { "enabled": true, "scaleFactor": 0.5 }
    }
  }
}
""")

# Nodes mit Farbe und Info
for n in G.nodes:
    core = G.nodes[n].get('core_distance', 0)
    allele = G.nodes[n].get('allele_distance', 0)
    title = f"Core: {core:.2f}, Allele: {allele:.2f}"
    label = f"{n}\n({core:.2f}, {allele:.2f})"

    # Default-Farbe
    color = "lightblue"

    if n in parental_nodes_hgt_events_corrected:
        color = "red"
    elif n < 100 and gene_absence_presence_matrix[n] == 1:
        color = "green"
    elif n < 100 and gene_absence_presence_matrix[n] == 0:
        color = "black"

    net.add_node(n, label=label, title=title, color=color)

# Kanten (gerichtet)
for u, v in G.edges:
    net.add_edge(u, v)

# Visualisierung anzeigen
net.show("graph.html", notebook = False)


graph.html


In [19]:
G.nodes[67]["level"]

0

In [34]:
from pyvis.network import Network
from collections import defaultdict

# 1. Netzwerk initialisieren
net = Network(height="900px", width="100%", directed=True)
net.set_options("""
{
  "layout": {
    "hierarchical": {
      "enabled": true,
      "direction": "UD",
      "sortMethod": "directed",
      "levelSeparation": 100,
      "nodeSpacing": 150,
      "treeSpacing": 200
    }
  },
  "nodes": {
    "shape": "dot",
    "size": 12,
    "font": { "size": 16 }
  },
  "edges": {
    "arrows": {
      "to": { "enabled": true, "scaleFactor": 0.5 }
    }
  }
}
""")

# 2. Gruppen von Nodes pro Level sammeln
levels = defaultdict(list)
for n in G.nodes:
    level = G.nodes[n].get("level", 0)
    levels[level].append(n)

# 3. Innerhalb eines Levels sortieren – Beispiel: nach ID
def subtree_size(node):
    return len(nx.descendants(G, node))

level_nodes.sort(key=lambda n: subtree_size(n))
# 4. Nodes einfügen mit Level-Angabe
for level, nodes_in_level in levels.items():
    for i, n in enumerate(nodes_in_level):
        core = G.nodes[n].get('core_distance', 0)
        allele = G.nodes[n].get('allele_distance', 0)
        title = f"Core: {core:.2f}, Allele: {allele:.2f}"
        label = f"{n}\n({core:.2f}, {allele:.2f})"

        color = "lightblue"
        if n in parental_nodes_hgt_events_corrected:
            color = "red"
        elif n < 100 and gene_absence_presence_matrix[n] == 1:
            color = "green"
        elif n < 100 and gene_absence_presence_matrix[n] == 0:
            color = "black"

        # wichtig: Level explizit setzen für hierarchische Darstellung
        net.add_node(n, label=label, title=title, color=color, level=level)

# 5. Edges hinzufügen
for u, v in G.edges:
    net.add_edge(u, v)

# 6. Anzeigen
net.show("graph.html", notebook=False)



graph.html


In [38]:
from pyvis.network import Network
from collections import defaultdict
import networkx as nx

# Netzwerk initialisieren
net = Network(height="900px", width="100%", directed=True)
net.set_options("""
{
  "layout": {
    "hierarchical": {
      "enabled": true,
      "direction": "UD",
      "sortMethod": "directed",
      "levelSeparation": 100,
      "nodeSpacing": 150,
      "treeSpacing": 200
    }
  },
  "nodes": {
    "shape": "dot",
    "size": 12,
    "font": { "size": 16 }
  },
  "edges": {
    "arrows": {
      "to": { "enabled": true, "scaleFactor": 0.5 }
    }
  }
}
""")

# Gruppiere Nodes nach Level
levels = defaultdict(list)
for n in G.nodes:
    level = G.nodes[n].get("level", 0)
    levels[level].append(int(n))

def descendants_count(node):
    return len(nx.descendants(G, node))

# Level sortieren, damit Level 0 zuerst kommt, dann aufsteigend
for level in sorted(levels.keys()):
    nodes_in_level = levels[level]

    if level == 0:
        # Nur Blätter mit n < 100 und numerisch sortieren
        leaves_level_0 = [n for n in nodes_in_level if n < 100]
        nodes_sorted = sorted(leaves_level_0)
    #else:
        # Andere Levels: sortiere nach Anzahl Nachkommen
    #    nodes_sorted = sorted(nodes_in_level, key=descendants_count)
    nodes_sorted = nodes_in_level
    
    for n in nodes_sorted:
        print(n)
        core = G.nodes[n].get('core_distance', 0)
        allele = G.nodes[n].get('allele_distance', 0)
        title = f"Core: {core:.2f}, Allele: {allele:.2f}"
        label = f"{n}\n({core:.2f}, {allele:.2f})"

        color = "lightblue"
        if n in parental_nodes_hgt_events_corrected:
            color = "red"
        elif n < 100 and gene_absence_presence_matrix[n] == 1:
            color = "green"
        elif n < 100 and gene_absence_presence_matrix[n] == 0:
            color = "black"

        net.add_node(n, label=label, title=title, color=color, level=level)

# Edges hinzufügen
for u, v in G.edges:
    net.add_edge(u, v)

net.show("graph.html", notebook=False)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
112
114
113
116
120
119
122
121
129
128
132
131
134
140
151
155
159
161
163
166
111
117
115
118
123
125
124
130
135
138
147
146
149
148
152
160
164
169
173
187
126
127
137
136
141
143
150
154
165
178
185
191
133
139
144
145
158
162
167
176
180
192
142
153
156
168
179
183
157
171
177
175
170
172
181
188
174
182
193
186
184
195
190
189
197
194
196
198
graph.html


In [36]:
levels

defaultdict(list,
            {0: [0,
              1,
              2,
              3,
              4,
              5,
              6,
              7,
              8,
              9,
              10,
              11,
              12,
              13,
              14,
              15,
              16,
              17,
              18,
              19,
              20,
              21,
              22,
              23,
              24,
              25,
              26,
              27,
              28,
              29,
              30,
              31,
              32,
              33,
              34,
              35,
              36,
              37,
              38,
              39,
              40,
              41,
              42,
              43,
              44,
              45,
              46,
              47,
              48,
              49,
              50,
              51,
              52,
              53,
              54,